In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using NurseData
using ForecastData
using GeographicData

In [4]:
using ReusableResourceAllocation

In [5]:
ne_states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0;

In [6]:
nurses = num_nurses_from_beds(ne_states);
demand = compute_ihme_forecast(start_date, end_date, ne_states);
adj = compute_adjacencies(ne_states, hrs=travel_threshold_hours);

In [7]:
model = reusable_resource_allocation(
    nurses,
    zeros(Float32, size(demand)...),
    demand,
    adj,
    send_new_only=false,
    send_wait_period=5,
    min_send_amt=10,
    smoothness_penalty=0.01,
    setup_cost=0.01,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 12175 rows, 8547 columns and 1324811 nonzeros
Model fingerprint: 0xbaa54934
Model has 385 SOS constraints
Variable types: 4741 continuous, 55 integer (55 binary)
Semi-Variable types: 3751 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+04]
Presolve removed 2723 rows and 2634 columns
Presolve time: 1.71s
Presolved: 11764 rows, 8021 columns, 338841 nonzeros
Presolved model has 2268 SOS constraint(s)
Variable types: 5845 continuous, 2176 integer (2176 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    4954    9.7503701e+04   1.383616e+11   0.000000e+00      5s
   10884    2.0875339e+05   3.512698e+09   0.000000e+00     10s
   15774    2.9859727e+0

In [8]:
shortage_per_day = (i,t) -> max.(0,
    demand[i,t] - (
        nurses[i]
        - sum(sent[i,:,1:t])
        + sum(sent[:,i,1:t-1])
    )
)
shortage = i -> sum(shortage_per_day(i,t) for t=1:size(sent,3))
total_shortage = sum(shortage(i) for i=1:size(sent,1))
@show total_shortage;

total_shortage = 10426.88219094655


In [9]:
summary = DataFrame(
    state=ne_states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    shortage=shortage.(1:length(ne_states)),
)

,state,total_sent,total_received,shortage
,String,Float64,Float64,Float64
1,CT,0.0,3473.88,2573.51
2,DE,460.914,0.0,0.544281
3,MA,2117.34,2869.54,1871.65
4,MD,1453.98,0.0,51.4058
5,ME,556.202,0.0,0.127243
6,NH,500.924,0.0,2.29179
7,NJ,0.0,3806.59,2070.65
8,NY,6911.35,6060.34,3133.83
9,PA,4436.89,496.141,161.371


In [10]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(ne_states))
insertcols!(sent_matrix, 1, :state => ne_states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,460.914,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2117.34,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,1453.98,0.0,0.0,0.0,0.0
5,ME,556.202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.924,0.0,0.0,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NY,2917.68,0.0,2869.54,0.0,0.0,0.0,0.0,0.0,496.141,627.986,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,1779.57,2657.32,0.0,0.0,0.0


In [11]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(nurses - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(ne_states));
# CSV.write("nurse_sent_matrix.csv", sent_vis_matrix);

In [12]:
n_days = size(sent,3)
outcomes = DataFrame()
for (i,s) in enumerate(ne_states)
    single_state_outcome = DataFrame(
        state=fill(s, n_days),
        day=start_date .+ Dates.Day.(0:n_days-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        total_nurses=[nurses[i] - sum(sent[i,:,1:t]) + sum(sent[:,i,1:t]) for t in 1:n_days],
        demand=demand[i,:],
        shortage=[shortage_per_day(i,t) for t in 1:n_days],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(ne_states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:n_days],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(ne_states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:n_days],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("nurse_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,total_nurses,demand,shortage,sent_to,sent_from
,String,Date,Float64,Float64,Float64,Float32,Float64,Any,Any
1,CT,2020-04-01,0.0,39.7287,2291.4,785.724,0.0,[],"[(""ME"", 39.7287)]"
2,DE,2020-04-01,92.1828,0.0,516.817,47.4493,0.0,"[(""NY"", 92.1828)]",[]
3,MA,2020-04-01,423.468,0.0,4005.2,597.547,0.0,"[(""NY"", 423.468)]",[]
4,MD,2020-04-01,181.748,0.0,2669.59,575.461,0.0,"[(""NJ"", 181.748)]",[]
5,ME,2020-04-01,39.7287,0.0,599.605,35.5339,0.0,"[(""CT"", 39.7287)]",[]
6,NH,2020-04-01,100.185,0.0,541.482,30.0956,0.0,"[(""NY"", 100.185)]",[]
7,NJ,2020-04-01,0.0,491.936,4491.27,3880.68,0.0,[],"[(""MD"", 181.748), (""PA"", 238.559), (""RI"", 71.6291)]"
8,NY,2020-04-01,0.0,1212.07,11203.4,10525.8,534.425,[],"[(""DE"", 92.1828), (""MA"", 423.468), (""NH"", 100.185), (""PA"", 531.464), (""VT"", 64.7688)]"
9,PA,2020-04-01,770.023,0.0,7457.31,892.099,0.0,"[(""NJ"", 238.559), (""NY"", 531.464)]",[]


In [13]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,total_nurses,demand,shortage,sent_to,sent_from
,String,Date,Float64,Float64,Float64,Float32,Float64,Any,Any
1,NJ,2020-04-01,0.0,491.936,4491.27,3880.68,0.0,[],"[(""MD"", 181.748), (""PA"", 238.559), (""RI"", 71.6291)]"
2,NJ,2020-04-02,0.0,491.936,4983.21,4655.39,164.122,[],"[(""MD"", 181.748), (""PA"", 238.559), (""RI"", 71.6291)]"
3,NJ,2020-04-03,0.0,491.936,5475.14,5475.14,491.936,[],"[(""MD"", 181.748), (""PA"", 238.559), (""RI"", 71.6291)]"
4,NJ,2020-04-04,0.0,466.156,5941.3,5703.39,228.244,[],"[(""MD"", 181.748), (""PA"", 212.779), (""RI"", 71.6291)]"
5,NJ,2020-04-05,0.0,466.156,6407.45,6188.15,246.855,[],"[(""MD"", 181.748), (""PA"", 212.779), (""RI"", 71.6291)]"
6,NJ,2020-04-06,0.0,466.156,6873.61,6613.42,205.968,[],"[(""MD"", 181.748), (""PA"", 212.779), (""RI"", 71.6291)]"
7,NJ,2020-04-07,0.0,466.156,7339.77,7079.6,205.992,[],"[(""MD"", 181.748), (""PA"", 212.779), (""RI"", 71.6291)]"
8,NJ,2020-04-08,0.0,466.156,7805.92,7814.22,474.456,[],"[(""MD"", 181.748), (""PA"", 212.779), (""RI"", 71.6291)]"
9,NJ,2020-04-09,0.0,0.0,7805.92,7859.0,53.0757,[],[]


In [14]:
println("Connectivity:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(adj)))

Connectivity:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["CT", "DE", "MA", "ME", "NH", "NJ", "NY", "VT"]
  "NJ" => ["CT", "DE", "MA", "MD", "NY", "PA", "RI"]
  "DE" => ["CT", "MD", "NJ", "NY", "PA", "RI"]
  "MD" => ["DE", "NJ", "NY", "PA"]
  "NH" => ["CT", "MA", "ME", "NY", "RI"]
  "NY" => ["CT", "DE", "MA", "MD", "NH", "NJ", "PA", "RI", "VT"]
  "ME" => ["CT", "MA", "NH", "RI", "VT"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "MA" => ["CT", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "PA" => ["DE", "MD", "NJ", "NY"]
  "VT" => ["CT", "MA", "ME", "NY", "RI"]

In [15]:
println("Sent to:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["NJ"]
  "NJ" => String[]
  "DE" => ["NY"]
  "MD" => ["NJ"]
  "NH" => ["NY"]
  "NY" => ["CT", "MA", "PA", "RI"]
  "ME" => ["CT"]
  "CT" => String[]
  "MA" => ["NY"]
  "PA" => ["NJ", "NY"]
  "VT" => ["NY"]